In [1]:
#https://keras.io/examples/generative/vae/
#https://keras.io/examples/generative/molecule_generation/

In [2]:
import os
from sklearn.preprocessing import MinMaxScaler
import joblib
import pandas as pd
import h5py
import tensorflow as tf
from IPython.display import clear_output,display, HTML
#================== initialization ==================
LATENT_DIM = 128
VAE_LR = 5e-4
PARQUET_PATH = './data/OptionsEOD_STG.parquet'
SCALER_PATH = './data/scaler.gz'
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']
SCALER_COL  = ['DTE','INTRINSIC_VALUE', 'TOTAL_VOLUME',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME' ]
H5_PATH = './data/OptTrainData.h5'
EPOCHS = 5

2024-06-23 06:46:12.373040: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#Example
import random
from IPython.display import clear_output,display, HTML
import numpy as np
#load scaler
scaler = MinMaxScaler()
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
random.shuffle(PartitionDate)
scaler = joblib.load(SCALER_PATH)

DATA  = np.empty((0,) + (20,9) )
for i,partdate in enumerate(PartitionDate) :
    df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                 , filters=[('PartitionDate', '=', partdate)]
                                )
    df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
    df['C_VOLUME'] = df['C_VOLUME'].fillna(0)

    for opt_id in np.unique( df[["OPTIONS_ID"]].values)[:128]:
        df_filter  = df[df["OPTIONS_ID"]==opt_id]
        if len(df_filter) == 20:
            DATA = np.vstack((DATA ,[scaler.transform(df_filter[SCALER_COL])]))
        else:
            #print( len(df_filter) )
            #display(HTML(df_filter[['STRIKE']+SCALER_COL].to_html()))
            pass
            
    break
        
#df_filter[SCALER_COL]


In [4]:
DATA.shape

(118, 20, 9)

In [5]:
# Save the NumPy array to an HDF5 file
with h5py.File(H5_PATH, 'w') as f:
    #dset = f.create_dataset('dataset', data=DATA, chunks=True, compression='gzip')
    #test
    dset = f.create_dataset('dataset', data=DATA, compression='gzip')

In [6]:
# เปิดไฟล์ HDF5
with h5py.File(H5_PATH, 'r') as f:
    # สร้าง dataset ใหม่โดยกำหนด chunk_size
    DATA = f['dataset'][:]

In [7]:
DATA.shape

(118, 20, 9)

In [8]:
X = DATA[:, :, :3]  # เลือกข้อมูลแถวแรกถึงแถวที่ 3 สำหรับ X
Y = DATA[:, :, 3:]  # เลือกข้อมูลแถวที่ 3 เป็นต้นไปสำหรับ Y

In [9]:
X.shape

(118, 20, 3)

In [10]:
#====================================================================
# gconv_units  last is latent_dim

In [11]:
from src.model import OptionChainGenerator
from src.layer import encoder, decoder

model = OptionChainGenerator(
    encoder(latent_dim = LATENT_DIM, 
            input_shape= (20,3), 
            dense_units = [128,64], 
            dropout_rate= 0.1
           ), 
    decoder(latent_dim  = LATENT_DIM , 
            output_shape= (20,1),
            dense_units = [128, 256],
            dropout_rate= 0.1
           )
)

def dummy_loss(y_true, y_pred):
    return 0.0
    
vae_optimizer = tf.keras.optimizers.Adam(learning_rate=VAE_LR)
model.compile(vae_optimizer , loss=dummy_loss)

In [12]:
model.summary()

Model: "option_chain_generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder (Functional)            │ ?                      │        25,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ ?                      │        80,376 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 105,784 (413.22 KB)

 Trainable params: 105,784 (413.22 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
#================== train model ==================
history = model.fit(X , Y, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5
i
i
i
i
i
i
A
i
i
i
i
i
i
A
1/3 ━━━━━━━━━━━━━━━━━━━━ 7s 4s/step - kl_loss: 0.0715 - optVal_loss: 145.4526 - total_loss: 205.0833 - vol_loss: 59.5593

ValueError: No loss to compute. Provide a `loss` argument in `compile()`.

In [ ]:
Y

In [ ]:
X.shape

In [ ]:
import numpy as np

def create_adjacency_matrix(options, threshold=25):
    """
    สร้าง adjacency matrix สำหรับ Options Chain โดยใช้ความใกล้เคียงของ strike price และ DTE
    """
    num_options = len(options)
    adjacency_matrix = np.zeros((num_options, num_options))

    for i in range(num_options):
        for j in range(i, num_options):
            # พิจารณาเชื่อมโยงระหว่าง options หาก strike price ต่างกันไม่เกิน threshold และ DTE เท่ากัน
            if abs(options[i]['strike_price'] - options[j]['strike_price']) <= threshold and options[i]['dte'] == options[j]['dte']:
                adjacency_matrix[i, j] = 1
                adjacency_matrix[j, i] = 1
    
    return adjacency_matrix

options = [
    {'strike_price': 100, 'dte': 30},
    {'strike_price': 105, 'dte': 30},
    {'strike_price': 110, 'dte': 30},
    {'strike_price': 115, 'dte': 30},
    {'strike_price': 120, 'dte': 30},
    {'strike_price': 0, 'dte': 0},  # เปลี่ยนค่าเป็น 0
]

options = [
    {'strike_price': 0, 'dte': 0},
    {'strike_price': 120, 'dte': 30},
    {'strike_price': 150, 'dte': 30},
    {'strike_price': 110, 'dte': 30},
    {'strike_price': 105, 'dte': 30},
    {'strike_price': 100, 'dte': 30},  # เปลี่ยนค่าเป็น 0
]


# ปรับ threshold
threshold = 5

adjacency_matrix = create_adjacency_matrix(options, threshold)
print(adjacency_matrix)

In [ ]:
import numpy as np
A = np.array(
    [[1,2,3],
    [4,5,6],
    [7,8,9]]
    
)

In [ ]:
A[:,:,0]

In [ ]:
import numpy as np

# Generate a random numpy array with shape (2, 20, 6)
random_array = np.random.rand(2, 20, 6)

print(random_array)

In [ ]:
random_array = np.random.rand(2, 5, 3)

In [ ]:
random_array

In [ ]:
random_array[:, :, -1]

In [ ]:
colList = ["c_bid", "c_ask", "c_volume", "p_bid", "p_ask", "p_volume"]

In [ ]:
colList.index("c_ask")